In [46]:
import pandas as pd
import numpy as np

In [47]:
# LOAD THE DATA AND RENAME THE COLUMN
# ACCOUNT CSV FILE
df = pd.read_csv("../data/accounts.csv")
df.rename(columns={'id': 'account_id'}, inplace=True)
df.rename(columns={'date': 'open_date'}, inplace=True)

In [48]:
# REMOVE SPACES FROM COLUMN NAMES
df.rename(columns=lambda x: x.strip(), inplace=True)

# COUNT THE NUMBER OF MISSING VALUES IN EACH COLUMN; none
missing_values_num = df.isna().sum()
print(missing_values_num)

account_id             0
district_id            0
open_date              0
statement_frequency    0
dtype: int64


In [49]:
# TYPE CAST THE DATE COLUMN
df['open_date'] = pd.to_datetime(df['open_date'])

/var/folders/1z/6nvrnkz13ms1y39fh2328y7m0000gn/T/ipykernel_74443/2923481415.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['open_date'] = pd.to_datetime(df['open_date'])


In [50]:
# LOAD DISTRCICT DATA AND GET THE DISTRICT NAME
district_df = pd.read_csv("./districts_py.csv")

df = pd.merge(df, district_df, on='district_id', how ='left')
df['district_id'] = df['name']

# for i in df.index:
#     for j in district_df.index: # convert it to python function
#         if (df['district_id'][i] == district_df['id'][j]):
#             df['district_id'][i] = district_df['name'][j]

In [51]:
# CHANGE THE COLUMN NAME AND DROP THE COLUMNS
df.rename(columns={'district_id': 'district_name'}, inplace=True)
df = df.iloc[:, :4]

In [52]:
# LOAD THE LINKS DATA FOR NUM_CUSTOMERS COLUMN
link_df = pd.read_csv("../data/links.csv")
temp = pd.DataFrame(link_df.groupby('account_id')['client_id'].count())
temp = temp.rename(columns = {'client_id':'num_customers'})
transpose = temp.T
temp['account'] = transpose.columns

df= df.merge(temp, left_on = "account_id", right_on = "account", how = "left")
df = df.drop("account", axis = 1)

In [53]:
# CREDIT_CARDS
card_df = pd.read_csv("../data/cards.csv")
temp = link_df.merge(card_df, left_on = "id", right_on = "link_id", how = "left")

temp = pd.DataFrame(temp.groupby("account_id")['id_y'].count())
temp = temp.rename(columns = {"id_y": "credit_cards"})

column_extract = temp.T

column_extract
temp['account'] = column_extract.columns

pd.unique(temp['credit_cards'])

array([0, 1])

In [54]:
df = df.merge(temp, left_on = "account_id", right_on="account", how = "left")
df.drop("account", axis = 1, inplace = True)
df

,account_id,district_name,open_date,statement_frequency,num_customers,credit_cards
0,1,Pisek,1995-03-24,monthly,1,0
1,2,Hl.m. Praha,1993-02-26,monthly,2,0
2,3,Kolin,1997-07-07,monthly,2,0
3,4,Pribram,1996-02-21,monthly,1,0
4,5,Cesky Krumlov,1997-05-30,monthly,1,0
...,...,...,...,...,...,...
4495,11027,Ceska Lipa,1996-07-26,weekly,1,0
4496,11054,Kromeriz,1996-02-01,weekly,1,0
4497,11138,Karvina,1995-08-18,weekly,1,1
4498,11141,Hl.m. Praha,1997-02-11,weekly,1,1


In [55]:
# LOAN
loan_df = pd.read_csv("./loans_py.csv")
df = pd.merge(df,loan_df, on='account_id', how='outer')
df['loan'] = df['loan_id'].apply(lambda x: 'True' if pd.notna(x) else 'False')
df

,account_id,district_name,open_date,statement_frequency,num_customers,credit_cards,loan_id,date,amount,payments,terms,status,loan
0,1,Pisek,1995-03-24,monthly,1,0,NaN,NaN,NaN,NaN,NaN,NaN,False
1,2,Hl.m. Praha,1993-02-26,monthly,2,0,4959.0,1994-01-05,80952.0,3373.0,24.0,A,True
2,3,Kolin,1997-07-07,monthly,2,0,NaN,NaN,NaN,NaN,NaN,NaN,False
3,4,Pribram,1996-02-21,monthly,1,0,NaN,NaN,NaN,NaN,NaN,NaN,False
4,5,Cesky Krumlov,1997-05-30,monthly,1,0,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,11027,Ceska Lipa,1996-07-26,weekly,1,0,7242.0,1997-10-23,48096.0,2004.0,24.0,C,True
4496,11054,Kromeriz,1996-02-01,weekly,1,0,7246.0,1996-08-20,148920.0,2482.0,60.0,C,True
4497,11138,Karvina,1995-08-18,weekly,1,1,7263.0,1996-01-19,89880.0,3745.0,24.0,A,True
4498,11141,Hl.m. Praha,1997-02-11,weekly,1,1,7264.0,1997-10-07,44940.0,3745.0,12.0,A,True


In [56]:
# LOAN_AMOUNT
df.rename(columns={'amount':'loan_amount'}, inplace=True)

# LOAN PAYMENT
df.rename(columns={'payments':'loan_payment'}, inplace=True)

df 

,account_id,district_name,open_date,statement_frequency,num_customers,credit_cards,loan_id,date,loan_amount,loan_payment,terms,status,loan
0,1,Pisek,1995-03-24,monthly,1,0,NaN,NaN,NaN,NaN,NaN,NaN,False
1,2,Hl.m. Praha,1993-02-26,monthly,2,0,4959.0,1994-01-05,80952.0,3373.0,24.0,A,True
2,3,Kolin,1997-07-07,monthly,2,0,NaN,NaN,NaN,NaN,NaN,NaN,False
3,4,Pribram,1996-02-21,monthly,1,0,NaN,NaN,NaN,NaN,NaN,NaN,False
4,5,Cesky Krumlov,1997-05-30,monthly,1,0,NaN,NaN,NaN,NaN,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,11027,Ceska Lipa,1996-07-26,weekly,1,0,7242.0,1997-10-23,48096.0,2004.0,24.0,C,True
4496,11054,Kromeriz,1996-02-01,weekly,1,0,7246.0,1996-08-20,148920.0,2482.0,60.0,C,True
4497,11138,Karvina,1995-08-18,weekly,1,1,7263.0,1996-01-19,89880.0,3745.0,24.0,A,True
4498,11141,Hl.m. Praha,1997-02-11,weekly,1,1,7264.0,1997-10-07,44940.0,3745.0,12.0,A,True


In [57]:
# LOAN_TERM
df.rename(columns={'terms':'loan_term'}, inplace=True)

# LOAN_STATUS
df['loan_status'] = df['status'].apply(lambda x: np.nan if pd.isna(x) else ("expired" if x == 'A' or x== 'B' else 'current')) # x: A, B, C, D, NaN / every row in loan_status

df

# if df['loan'] == 'T':
#     if loan_df['status'] == 'A' or loan_df['status'] == 'B':
#         df['loan_status'] = 'expired'
#     else: # C or D
#         df['loan_status'] = 'current'

,account_id,district_name,open_date,statement_frequency,num_customers,credit_cards,loan_id,date,loan_amount,loan_payment,loan_term,status,loan,loan_status
0,1,Pisek,1995-03-24,monthly,1,0,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
1,2,Hl.m. Praha,1993-02-26,monthly,2,0,4959.0,1994-01-05,80952.0,3373.0,24.0,A,True,expired
2,3,Kolin,1997-07-07,monthly,2,0,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
3,4,Pribram,1996-02-21,monthly,1,0,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
4,5,Cesky Krumlov,1997-05-30,monthly,1,0,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,11027,Ceska Lipa,1996-07-26,weekly,1,0,7242.0,1997-10-23,48096.0,2004.0,24.0,C,True,current
4496,11054,Kromeriz,1996-02-01,weekly,1,0,7246.0,1996-08-20,148920.0,2482.0,60.0,C,True,current
4497,11138,Karvina,1995-08-18,weekly,1,1,7263.0,1996-01-19,89880.0,3745.0,24.0,A,True,expired
4498,11141,Hl.m. Praha,1997-02-11,weekly,1,1,7264.0,1997-10-07,44940.0,3745.0,12.0,A,True,expired


In [58]:
# LOAN_DEFAULT:
df['loan_default'] = df['status'].apply(lambda x: np.nan if pd.isna(x) else ('True' if x == 'B' or x== 'D' else 'False')) # x: A, B, C, D, NaN / every row in loan_status

# if loan_df['status'] == 'B' or loan_df['status'] == 'D':
#     df['loan_default'] = 'T'
# else:
#     df['loan_default'] = 'F'

In [60]:
# MAX_WITHDRAWAL
trans_df = pd.read_csv("../data/transactions.csv")
trans_df.head()

# return the maximum value of each account

# maximum value of debit
temp = trans_df[trans_df['type'] == 'debit'] # filter out
temp.groupby('account_id')


/var/folders/1z/6nvrnkz13ms1y39fh2328y7m0000gn/T/ipykernel_74443/3443830624.py:2: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  trans_df = pd.read_csv("../data/transactions.csv")


In [61]:
print(temp.groupby('account_id')['amount'].max()) # Series 
max_with = temp.groupby('account_id')['amount'].max()
max_with = pd.DataFrame(max_with)
type(max_with)

account_id
1         7500
2        42000
3         7400
4         5250
5         5100
         ...  
11333    62800
11349    64800
11359    30100
11362    16100
11382    52800
Name: amount, Length: 4500, dtype: int64


pandas.core.frame.DataFrame

In [62]:
print(max_with)
df = pd.merge(df, max_with, on='account_id', how='left')
df.rename(columns = {'amount':'max_withdrawal'}, inplace=True)
df

            amount
account_id        
1             7500
2            42000
3             7400
4             5250
5             5100
...            ...
11333        62800
11349        64800
11359        30100
11362        16100
11382        52800

[4500 rows x 1 columns]


,account_id,district_name,open_date,statement_frequency,num_customers,credit_cards,loan_id,date,loan_amount,loan_payment,loan_term,status,loan,loan_status,loan_default,max_withdrawal
0,1,Pisek,1995-03-24,monthly,1,0,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,7500
1,2,Hl.m. Praha,1993-02-26,monthly,2,0,4959.0,1994-01-05,80952.0,3373.0,24.0,A,True,expired,False,42000
2,3,Kolin,1997-07-07,monthly,2,0,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,7400
3,4,Pribram,1996-02-21,monthly,1,0,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,5250
4,5,Cesky Krumlov,1997-05-30,monthly,1,0,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,5100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,11027,Ceska Lipa,1996-07-26,weekly,1,0,7242.0,1997-10-23,48096.0,2004.0,24.0,C,True,current,False,20320
4496,11054,Kromeriz,1996-02-01,weekly,1,0,7246.0,1996-08-20,148920.0,2482.0,60.0,C,True,current,False,22780
4497,11138,Karvina,1995-08-18,weekly,1,1,7263.0,1996-01-19,89880.0,3745.0,24.0,A,True,expired,False,64200
4498,11141,Hl.m. Praha,1997-02-11,weekly,1,1,7264.0,1997-10-07,44940.0,3745.0,12.0,A,True,expired,False,78600


In [63]:
print(temp.groupby('account_id')['amount'].min()) # Series 
min_with = temp.groupby('account_id')['amount'].min()
min_with = pd.DataFrame(min_with)
type(min_with)

account_id
1        15
2        15
3        15
4        15
5        15
         ..
11333     0
11349    30
11359    15
11362    15
11382     1
Name: amount, Length: 4500, dtype: int64


pandas.core.frame.DataFrame

In [64]:
print(min_with)
df = pd.merge(df, min_with, on='account_id', how='left')
df.rename(columns = {'amount':'min_withdrawal'}, inplace=True)
df

            amount
account_id        
1               15
2               15
3               15
4               15
5               15
...            ...
11333            0
11349           30
11359           15
11362           15
11382            1

[4500 rows x 1 columns]


,account_id,district_name,open_date,statement_frequency,num_customers,credit_cards,loan_id,date,loan_amount,loan_payment,loan_term,status,loan,loan_status,loan_default,max_withdrawal,min_withdrawal
0,1,Pisek,1995-03-24,monthly,1,0,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,7500,15
1,2,Hl.m. Praha,1993-02-26,monthly,2,0,4959.0,1994-01-05,80952.0,3373.0,24.0,A,True,expired,False,42000,15
2,3,Kolin,1997-07-07,monthly,2,0,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,7400,15
3,4,Pribram,1996-02-21,monthly,1,0,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,5250,15
4,5,Cesky Krumlov,1997-05-30,monthly,1,0,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,5100,15
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,11027,Ceska Lipa,1996-07-26,weekly,1,0,7242.0,1997-10-23,48096.0,2004.0,24.0,C,True,current,False,20320,30
4496,11054,Kromeriz,1996-02-01,weekly,1,0,7246.0,1996-08-20,148920.0,2482.0,60.0,C,True,current,False,22780,30
4497,11138,Karvina,1995-08-18,weekly,1,1,7263.0,1996-01-19,89880.0,3745.0,24.0,A,True,expired,False,64200,30
4498,11141,Hl.m. Praha,1997-02-11,weekly,1,1,7264.0,1997-10-07,44940.0,3745.0,12.0,A,True,expired,False,78600,30


In [69]:
## CC payments
temp = trans_df[(trans_df['type'] == 'debit') & (trans_df['method'] == 'credit card')] # filter out
temp = temp.groupby('account_id')['amount'].count()
temp

account_id
7         1
14        6
33       22
34        1
43       35
         ..
11079     4
11138     8
11141     5
11186     2
11359    22
Name: amount, Length: 807, dtype: int64

In [68]:
temp = pd.DataFrame(temp)
# print(df.shape)
# print(temp.shape)
df = pd.merge(df, temp, on='account_id', how='left')
df.rename(columns = {'amount':'cc_payments'}, inplace=True)
df

,account_id,district_name,open_date,statement_frequency,num_customers,credit_cards,loan_id,date,loan_amount,loan_payment,loan_term,status,loan,loan_status,loan_default,max_withdrawal,min_withdrawal,cc_payments
0,1,Pisek,1995-03-24,monthly,1,0,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,7500,15,NaN
1,2,Hl.m. Praha,1993-02-26,monthly,2,0,4959.0,1994-01-05,80952.0,3373.0,24.0,A,True,expired,False,42000,15,NaN
2,3,Kolin,1997-07-07,monthly,2,0,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,7400,15,NaN
3,4,Pribram,1996-02-21,monthly,1,0,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,5250,15,NaN
4,5,Cesky Krumlov,1997-05-30,monthly,1,0,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,5100,15,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,11027,Ceska Lipa,1996-07-26,weekly,1,0,7242.0,1997-10-23,48096.0,2004.0,24.0,C,True,current,False,20320,30,NaN
4496,11054,Kromeriz,1996-02-01,weekly,1,0,7246.0,1996-08-20,148920.0,2482.0,60.0,C,True,current,False,22780,30,NaN
4497,11138,Karvina,1995-08-18,weekly,1,1,7263.0,1996-01-19,89880.0,3745.0,24.0,A,True,expired,False,64200,30,8.0
4498,11141,Hl.m. Praha,1997-02-11,weekly,1,1,7264.0,1997-10-07,44940.0,3745.0,12.0,A,True,expired,False,78600,30,5.0


In [73]:
# MAX BALANCE
temp = trans_df.groupby('account_id')['balance'].max() # filter out
print(temp)

max_bal = pd.DataFrame(temp)

df = pd.merge(df, max_bal, on='account_id', how='left')
df.rename(columns = {'balance':'max_balance'}, inplace=True)
df

account_id
1         30415
2         69302
3         53447
4         34870
5         32036
          ...  
11333    137043
11349    113679
11359     81706
11362     56056
11382     93550
Name: balance, Length: 4500, dtype: int64


,account_id,district_name,open_date,statement_frequency,num_customers,credit_cards,loan_id,date,loan_amount,loan_payment,loan_term,status,loan,loan_status,loan_default,max_withdrawal,min_withdrawal,cc_payments,max_balance
0,1,Pisek,1995-03-24,monthly,1,0,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,7500,15,NaN,30415
1,2,Hl.m. Praha,1993-02-26,monthly,2,0,4959.0,1994-01-05,80952.0,3373.0,24.0,A,True,expired,False,42000,15,NaN,69302
2,3,Kolin,1997-07-07,monthly,2,0,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,7400,15,NaN,53447
3,4,Pribram,1996-02-21,monthly,1,0,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,5250,15,NaN,34870
4,5,Cesky Krumlov,1997-05-30,monthly,1,0,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,5100,15,NaN,32036
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,11027,Ceska Lipa,1996-07-26,weekly,1,0,7242.0,1997-10-23,48096.0,2004.0,24.0,C,True,current,False,20320,30,NaN,51010
4496,11054,Kromeriz,1996-02-01,weekly,1,0,7246.0,1996-08-20,148920.0,2482.0,60.0,C,True,current,False,22780,30,NaN,66825
4497,11138,Karvina,1995-08-18,weekly,1,1,7263.0,1996-01-19,89880.0,3745.0,24.0,A,True,expired,False,64200,30,8.0,168372
4498,11141,Hl.m. Praha,1997-02-11,weekly,1,1,7264.0,1997-10-07,44940.0,3745.0,12.0,A,True,expired,False,78600,30,5.0,144498


In [74]:
# MIN BALANCE
temp = trans_df.groupby('account_id')['balance'].min() # filter out
print(temp)

min_bal = pd.DataFrame(temp)

df = pd.merge(df, min_bal, on='account_id', how='left')
df.rename(columns = {'balance':'min_balance'}, inplace=True)
df

account_id
1        1000
2        1100
3        1000
4         800
5         600
         ... 
11333   -1291
11349     200
11359    1000
11362    1000
11382   -1037
Name: balance, Length: 4500, dtype: int64


,account_id,district_name,open_date,statement_frequency,num_customers,credit_cards,loan_id,date,loan_amount,loan_payment,loan_term,status,loan,loan_status,loan_default,max_withdrawal,min_withdrawal,cc_payments,max_balance,min_balance
0,1,Pisek,1995-03-24,monthly,1,0,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,7500,15,NaN,30415,1000
1,2,Hl.m. Praha,1993-02-26,monthly,2,0,4959.0,1994-01-05,80952.0,3373.0,24.0,A,True,expired,False,42000,15,NaN,69302,1100
2,3,Kolin,1997-07-07,monthly,2,0,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,7400,15,NaN,53447,1000
3,4,Pribram,1996-02-21,monthly,1,0,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,5250,15,NaN,34870,800
4,5,Cesky Krumlov,1997-05-30,monthly,1,0,NaN,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,5100,15,NaN,32036,600
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4495,11027,Ceska Lipa,1996-07-26,weekly,1,0,7242.0,1997-10-23,48096.0,2004.0,24.0,C,True,current,False,20320,30,NaN,51010,800
4496,11054,Kromeriz,1996-02-01,weekly,1,0,7246.0,1996-08-20,148920.0,2482.0,60.0,C,True,current,False,22780,30,NaN,66825,400
4497,11138,Karvina,1995-08-18,weekly,1,1,7263.0,1996-01-19,89880.0,3745.0,24.0,A,True,expired,False,64200,30,8.0,168372,800
4498,11141,Hl.m. Praha,1997-02-11,weekly,1,1,7264.0,1997-10-07,44940.0,3745.0,12.0,A,True,expired,False,78600,30,5.0,144498,200


In [76]:
columns_order = ['account_id', 'district_name', 'open_date','statement_frequency', 'num_customers', 'credit_cards', 'loan', 'loan_amount', 'loan_payment', 'loan_term', 'loan_status', 'loan_default','max_withdrawal','min_withdrawal','cc_payments','max_balance','min_balance']
df = df[columns_order]

In [77]:
df.to_csv('analytics_py.csv', index = False, encoding='utf-8') # open csv preview